Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle -Uqq
!mkdir /root/.kaggle
!mkdir model_checkpoints
!cp /content/drive/MyDrive/kaggle.json  /root/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle.json  /root/.kaggle

In [ ]:

!pip install wandb -Uqq

In [ ]:
!kaggle datasets download -d preetviradiya/brian-tumor-dataset

In [ ]:
!mkdir data
!unzip -d data -q /content/brian-tumor-dataset.zip
!rm -rf /content/brian-tumor-dataset.zip

In [ ]:
import pandas as pd

In [ ]:
metadata = pd.read_csv("/content/data/metadata.csv")
metadata_rgb = pd.read_csv("/content/data/metadata_rgb_only.csv")

In [ ]:
metadata

In [ ]:
metadata_rgb

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import wandb
from wandb.integration.keras import WandbCallback
from pathlib import Path

In [ ]:
tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
path = Path('/content/data/Brain Tumor Data Set/Brain Tumor Data Set')
batch_size = 32
img_height = 224
img_width = 224

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="both",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
tf.keras.backend.clear_session()

resnet_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling='avg',
)
rescaling = layers.Rescaling(1.0 / 255)
resize = layers.Resizing(img_height, img_width)

model = Sequential()
#rescale and resize the image
model.add(resize)
model.add(rescaling)

model.add(resnet_base)
model.add(layers.Dense(1, activation='sigmoid'))

#freeze the base resnet_base from model
resnet_base.trainable = False

In [ ]:
batch = next(iter(train_ds))
outputs = model(batch[0])
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
roc_auc = tf.keras.metrics.AUC(curve='ROC', name='roc_auc')
accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')

In [ ]:
# define callbacks
decay_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
checkpoint_filepath = "/content/model_checkpoints/models_resnet_50/model_{epoch:02d}_val_roc_auc_{val_roc_auc:.4f}_val_loss_{val_loss:.4f}_val_accuracy_{val_accuracy:.4f}.weights.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[roc_auc, accuracy])

In [ ]:
from google.colab import userdata
WANDB_API_KEY = userdata.get('wandb')

In [ ]:
wandb.login(key=WANDB_API_KEY)

In [ ]:
!wandb login --relogin


In [ ]:
import wandb

# Replace 'your-api-key' with your actual W&B API key
wandb.login(key='14490665281cfce57ebb93f7fbc543f719e0c014')

In [ ]:
run = wandb.init(project="cortex", entity="shivanikotian18", job_type="finetune", group='resnet',tags=['ResNet50'])
assert run is wandb.run, "Something went wrong"
#train and validate the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=[checkpoint, WandbCallback(save_weights_only=False)],
)
run.finish()

In [ ]:
tf.keras.backend.clear_session()
VGG_model = Sequential()

pretrained_model= keras.applications.VGG19(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',
                   weights='imagenet')

rescaling = layers.Rescaling(1.0 / 255)
resize = layers.Resizing(img_height, img_width)

#rescale and resize the image
VGG_model.add(resize)
VGG_model.add(rescaling)
VGG_model.add(pretrained_model)


VGG_model.add(Dense(1, activation='sigmoid'))
pretrained_model.trainable=False

In [ ]:
batch = next(iter(train_ds))
outputs = VGG_model(batch[0])
VGG_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
roc_auc = tf.keras.metrics.AUC(curve='ROC', name='roc_auc')
accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')

In [ ]:
VGG_model.compile(optimizer=optimizer, loss=loss, metrics=[roc_auc, accuracy])

In [ ]:
# define callbacks
decay_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

checkpoint_filepath_vgg = "/content/model_checkpoints/models_vgg_19/model_{epoch:02d}_val_roc_auc_{val_roc_auc:.4f}_val_loss_{val_loss:.4f}_val_accuracy_{val_accuracy:.4f}.keras"
checkpoint_vgg = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_vgg,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
)

In [ ]:
run_vgg = wandb.init(project="combined", entity="shivanikotian18", job_type="finetune", group='vgg',tags=['VGG19'])
#train and validate the model
history = VGG_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=[checkpoint_vgg, WandbCallback(save_weights_only=False)],
)
run_vgg.finish()

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

In [ ]:
# Clear any previous model session to start fresh
tf.keras.backend.clear_session()

# Initialize a Sequential model
custom_model = Sequential()

# Add a 2D convolution layer with 16 filters, a 3x3 kernel, and 'relu' activation function
custom_model.add(Conv2D(16, (3, 3), activation='relu', padding = "same", input_shape=(224, 224, 3)))

# Add a max pooling layer to reduce the spatial dimensions of the output volume
custom_model.add(MaxPooling2D((2, 2)))

# Add another convolution and max pooling layer
custom_model.add(Conv2D(8, (3, 3), activation='relu', padding = "same"))
custom_model.add(MaxPool2D((2,2)))

# Flatten the tensor output from the previous layer to create a single long feature vector
custom_model.add(Flatten())

# Add a dense layer with 8 neurons and 'relu' activation function
custom_model.add(Dense(8, activation='relu'))

# Add batch normalization layer to normalize the activations of the previous layer
custom_model.add(BatchNormalization())

# Add a dropout layer to prevent overfitting
custom_model.add(Dropout(0.3))

# Add the output layer with a single neuron (for binary classification) and 'sigmoid' activation function
custom_model.add(Dense(1, activation='sigmoid'))


In [ ]:
batch = next(iter(train_ds))
outputs = custom_model(batch[0])
custom_model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
roc_auc = tf.keras.metrics.AUC(curve='ROC', name='roc_auc')
accuracy = tf.keras.metrics.BinaryAccuracy(name='accuracy')

custom_model.compile(optimizer=optimizer, loss=loss, metrics=[roc_auc, accuracy])

# define callbacks
decay_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

checkpoint_filepath_custom = "/content/model_checkpoints/models_custom/model_{epoch:02d}_val_roc_auc_{val_roc_auc:.4f}_val_loss_{val_loss:.4f}_val_accuracy_{val_accuracy:.4f}.keras"
checkpoint_custom = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_custom,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    save_freq="epoch",
)

In [ ]:
run_custom = wandb.init(project="cortex", entity="shivanikotian18", job_type="finetune", group='custom',tags=['Custom_model'])
#train and validate the model
history = custom_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=[checkpoint_custom, WandbCallback(save_weights_only=False)],
)
run_custom.finish()

In [ ]:
pip install wandb

In [ ]:
import wandb
from tensorflow.keras.models import load_model


In [ ]:
from tensorflow.keras.models import load_model

# The artifact directory will contain the saved model file
loaded_model = load_model(f"{artifact_dir}/model.h5")


In [ ]:
# If you don't have these installed
#!pip install pydot
#!apt-get install graphviz -y
from tensorflow.keras.utils import plot_model

# Create a diagram of the model architecture
#plot_model(VGG_model, to_file='VGG19model.png', show_shapes=True, show_layer_names=True)
plot_model(model, to_file='Resnet50Model.png', show_shapes=True, show_layer_names=True)
#plot_model(custom_model, to_file='HALNet_1_Model.png', show_shapes=True, show_layer_names=True)
from google.colab import files
files.download('VGG19_model.png')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import seaborn as sns

# Model paths and names
models_info = {
    "ResNet50": "/content/drive/MyDrive/Colab Notebooks/Final Codes/Resnet50.h5",
    "VGG19": "/content/drive/MyDrive/Colab Notebooks/Final Codes/VGG19.h5",
    "HALNet1": "/content/drive/MyDrive/Colab Notebooks/Final Codes/HALNet1.h5",
    "HALNet2": "/content/drive/MyDrive/Colab Notebooks/Final Codes/HALNet2.h5",
    "SMVNet": "/content/drive/MyDrive/Colab Notebooks/Final Codes/SMVNet.h5",
    "JAPNet": "/content/drive/MyDrive/Colab Notebooks/Final Codes/JAPnet.h5"
}

# Function to load model and get predictions
def get_model_predictions(model, dataset):
    predictions = []
    labels = []
    for x, y in dataset:
        preds = model.predict(x)
        predictions.extend(preds.flatten())  # Make sure to flatten or adjust depending on model output
        labels.extend(y.numpy())
    return np.array(predictions), np.array(labels)

# Prepare to collect ROC data
roc_data = []

# Process each model
for model_name, model_path in models_info.items():
    # Load the model
    model = load_model(model_path)

    # Get predictions and labels
    predictions, labels = get_model_predictions(model, val_ds)

    threshold = 0.5
    predicted_labels = (predictions > threshold).astype(int)
    # Compute confusion matrix
    cm = confusion_matrix(labels, predicted_labels)
    # Compute other metrics
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()

    # Print metrics
    print(f"Metrics for {model_name}:")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1_score:.2f}")
    print("------\n")


    # Calculate ROC AUC
    fpr, tpr, _ = roc_curve(labels, predictions)
    auc = roc_auc_score(labels, predictions)

    # Store the ROC data
    roc_data.append((fpr, tpr, auc, model_name))

# Plot all ROC curves in one figure
plt.figure(figsize=(10, 8))
for fpr, tpr, auc, model_name in roc_data:
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.2f})')

# Add the random chance line
plt.plot([0, 1], [0, 1], 'k--')

# Customize the plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Comparison of ROC Curves Across Models')
plt.legend(loc='lower right')
plt.show()
